# WRDS CRSP Documentation

### Treasury Libraries
`crsp_m_treasuries`

Quote Data
* tfz_dly: main file of treasury quotes
* tfz_dly_ts2: supplemental series
* tfz_dly_ft: matched to term structure at any given date
* tfz_dly_cd: compilation of fed reported rates in tfz format
* tfz_dly_cpi: compilation of cpi data
* tfz_dly_rf2: compiled risk-free rate

Identifying Info
* tfz_iss: issue details (which are included automatically in queries.)
* tfz_idx: description of KYTREASNOX codes

### WRDS Python Manual
https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/

https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/python-example-data-workflow/

https://wrds-www.wharton.upenn.edu/pages/support/applications/python-replications/fama-french-factors-python/

### CRSP Treasury Manual

https://www.crsp.org/products/documentation/treasno-properties
https://www.crsp.org/products/documentation/reference-information

In [1]:
import wrds
file_key = open("../../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

# Library Info

In [3]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')
display(treasury_databases)

['tfz_dly',
 'tfz_dly_cd',
 'tfz_dly_cpi',
 'tfz_dly_ft',
 'tfz_dly_rf2',
 'tfz_dly_ts2',
 'tfz_idx',
 'tfz_iss',
 'tfz_mast',
 'tfz_mth',
 'tfz_mth_bp',
 'tfz_mth_cd',
 'tfz_mth_cpi',
 'tfz_mth_fb',
 'tfz_mth_ft',
 'tfz_mth_rf',
 'tfz_mth_rf2',
 'tfz_mth_ts',
 'tfz_mth_ts2',
 'tfz_pay']

In [4]:
db.describe_table(library='crsp_m_treasuries', table='tfz_dly')

Approximately 3213036 rows in crsp_m_treasuries.tfz_dly.


,name,nullable,type
0,kytreasno,True,DOUBLE_PRECISION
1,kycrspid,True,VARCHAR(15)
2,caldt,True,DATE
3,tdbid,True,DOUBLE_PRECISION
4,tdask,True,DOUBLE_PRECISION
5,tdnomprc,True,DOUBLE_PRECISION
6,tdnomprc_flg,True,VARCHAR(1)
7,tdsourcr,True,VARCHAR(1)
8,tdaccint,True,DOUBLE_PRECISION
9,tdretnua,True,DOUBLE_PRECISION


# Database quote

In [8]:
DATE = '2022-06-30'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg


In [7]:
KEYS = tuple(data_quotes['kytreasno'])
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204072.0,20200215.108500,20200215.108500,912810EE,1990-02-15,2020-02-15,1.0,8.500,2.0,4.2500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204073.0,20200515.108750,20200515.108750,912810EF,1990-05-15,2020-05-15,1.0,8.750,2.0,4.3750,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204074.0,20200815.108750,20200815.108750,912810EG,1990-08-15,2020-08-15,1.0,8.750,2.0,4.3750,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204075.0,20210215.107870,20210215.107870,912810EH,1991-02-15,2021-02-15,1.0,7.875,2.0,3.9375,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204076.0,20210515.108120,20210515.108120,912810EJ,1991-05-15,2021-05-15,1.0,8.125,2.0,4.0625,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,207455.0,20201231.400000,20201231.400000,912796TY,2020-01-02,2020-12-31,1.0,0.000,0.0,0.0000,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
395,207456.0,20211231.201620,20211231.201620,912828YZ,2019-12-31,2021-12-31,1.0,1.625,2.0,0.8125,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
396,207457.0,20221215.201620,20221215.201620,912828YW,2019-12-15,2022-12-15,0.0,1.625,2.0,0.8125,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
397,207458.0,20241231.201750,20241231.201750,912828YY,2019-12-31,2024-12-31,0.0,1.750,2.0,0.8750,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None


In [8]:
data = data_quotes.merge(data_iss, on='kytreasno')
data

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204074.0,20200815.108750,2019-12-31,104.335938,104.359375,104.347656,M,I,3.281250,0.000076,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204072.0,20200215.108500,2019-12-31,100.750000,100.773438,100.761719,M,I,3.187500,-0.000229,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204073.0,20200515.108750,2019-12-31,102.632812,102.656250,102.644531,M,I,1.105769,-0.000295,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204075.0,20210215.107870,2019-12-31,106.812500,106.835938,106.824219,M,I,2.953125,-0.000659,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204076.0,20210515.108120,2019-12-31,108.875000,108.898438,108.886719,M,I,1.026786,0.000630,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,207451.0,20200303.400000,2019-12-31,99.732250,99.734000,99.733125,M,I,0.000000,NaN,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
395,207452.0,20200611.400000,2019-12-31,99.316306,99.320833,99.318569,M,I,0.000000,0.000180,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
396,207453.0,20200625.400000,2019-12-31,99.242833,99.247750,99.245292,M,I,0.000000,0.000068,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
397,207454.0,20200702.400000,2019-12-31,99.205222,99.212889,99.209056,M,I,0.000000,0.000043,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


# Get Yield Curve Data (CRSP)

In [9]:
data_ft = db.get_table(library='crsp_m_treasuries', table='tfz_dly_ft').pivot_table(index='caldt',values='tdytm', columns='kytreasnox')
data_ft.columns = [1,2,5,7,10,20,30]
data_ft

,1,2,5,7,10,20,30
caldt,,,,,,,
1961-06-14,2.935907,3.360687,3.623677,3.768720,3.818819,3.814210,3.815172
1961-06-15,2.932936,3.376460,3.671691,3.804225,3.862987,3.828220,3.826316
1961-06-16,2.929949,3.375670,3.685431,3.804216,3.863282,3.832922,3.830049
1961-06-19,2.920884,3.389970,3.712984,3.824557,3.886205,3.842378,3.837543
1961-06-20,2.952419,3.355796,3.685391,3.809274,3.886506,3.856465,3.845018
...,...,...,...,...,...,...,...
2022-04-25,2.004627,2.629813,2.863810,2.842773,2.800729,3.060412,2.867910
2022-04-26,1.948537,2.473193,2.720330,2.728669,2.702851,2.990138,2.806869
2022-04-27,1.909012,2.550330,2.808044,2.832539,2.812572,3.090740,2.901020


# Get Timeseries for certain maturities

In [24]:
MATDT = '2022-02-15'

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where tmatdt=\'{MATDT}\''
temp = db.raw_sql(SQL_QUERY)
#temp = temp[temp['itype'].isin([11,12]) == DOTIPS]
temp

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,206339.0,20220215.202000,20220215.202000,912828SF,2012-02-15,2022-02-15,1.0,2.0,2.0,1.00,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
1,207309.0,20220215.202500,20220215.202500,9128286C,2019-02-15,2022-02-15,1.0,2.5,2.0,1.25,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
2,207781.0,20220215.400000,20220215.400000,912796R8,2021-10-19,2022-02-15,1.0,0.0,0.0,0.00,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


In [25]:
if temp.shape[0]>1:
    KEYNO = tuple(temp['kytreasno'])
    SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO}'
else:
    KEYNO = temp['kytreasno'].values[0]
    SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno =\'{KEYNO}\''
    
ts = db.raw_sql(SQL_QUERY)
ts

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,206339.0,20220215.202000,2012-02-09,99.640625,99.703125,99.671875,M,I,0.0,NaN,0.000055,3334.278135,NaN,NaN,0.0,None,None
1,206339.0,20220215.202000,2012-02-10,100.093750,100.140625,100.117188,M,I,0.0,0.004468,0.000054,3334.172369,NaN,NaN,0.0,None,None
2,206339.0,20220215.202000,2012-02-13,100.187500,100.250000,100.218750,M,I,0.0,0.001014,0.000054,3331.345227,NaN,NaN,0.0,None,None
3,206339.0,20220215.202000,2012-02-14,100.546875,100.593750,100.570312,M,I,0.0,0.003508,0.000053,3331.044132,NaN,NaN,0.0,None,None
4,206339.0,20220215.202000,2012-02-15,100.625000,100.687500,100.656250,M,I,0.0,0.000855,0.000053,3330.206381,NaN,NaN,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,207781.0,20220215.400000,2022-01-25,99.997083,99.997667,99.997375,M,I,0.0,0.000007,0.000001,21.000000,NaN,61711.0,0.0,None,None
3347,207781.0,20220215.400000,2022-01-26,99.997222,99.997778,99.997500,M,I,0.0,0.000001,0.000001,20.000000,NaN,61711.0,0.0,None,None
3348,207781.0,20220215.400000,2022-01-27,99.997361,99.997889,99.997625,M,I,0.0,0.000001,0.000001,19.000000,NaN,61711.0,0.0,None,None
3349,207781.0,20220215.400000,2022-01-28,99.997750,99.998250,99.998000,M,I,0.0,0.000004,0.000001,18.000000,NaN,61711.0,0.0,None,None


# Get Timeseries for certain issues

In [26]:
KEYNO_ts = (207816,207818)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO_ts}'

ts_issue = db.raw_sql(SQL_QUERY)
ts_issue

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,207818.0,20221229.400000,2021-12-27,99.617708,99.627903,99.622806,M,I,0.0,NaN,0.000010,367.0,None,NaN,0.0,None,None
1,207816.0,20220630.400000,2022-02-03,99.869333,99.873417,99.871375,M,I,0.0,-0.000002,0.000009,147.0,None,58198.0,0.0,None,None
2,207816.0,20220630.400000,2022-02-04,99.837778,99.854000,99.845889,M,I,0.0,-0.000255,0.000011,146.0,None,58198.0,0.0,None,None
3,207816.0,20220630.400000,2022-02-07,99.851042,99.855014,99.853028,M,I,0.0,0.000071,0.000010,143.0,None,58198.0,0.0,None,None
4,207816.0,20220630.400000,2022-02-08,99.850111,99.854056,99.852083,M,I,0.0,-0.000009,0.000010,142.0,None,58198.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,207818.0,20221229.400000,2022-01-20,99.480736,99.495028,99.487882,M,I,0.0,-0.000057,0.000015,343.0,None,38798.0,0.0,None,None
170,207818.0,20221229.400000,2022-01-21,99.472750,99.487000,99.479875,M,I,0.0,-0.000080,0.000015,342.0,None,38798.0,0.0,None,None
171,207818.0,20221229.400000,2022-01-24,99.477375,99.486792,99.482083,M,I,0.0,0.000022,0.000015,339.0,None,38798.0,0.0,None,None
172,207818.0,20221229.400000,2022-01-25,99.446056,99.455444,99.450750,M,I,0.0,-0.000315,0.000016,338.0,None,38798.0,0.0,None,None


# Get Timeseries for TIPS issues

In [33]:
KEYNO_tips = (207431,206999,207009)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNO_tips}'

ts_issue_tips = db.raw_sql(SQL_QUERY)
ts_issue_tips

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,206999.0,20260715.C00120,2016-07-14,101.171875,101.203125,101.187500,M,I,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.99987,C
1,206999.0,20260715.C00120,2019-06-24,99.421875,99.523438,99.472656,M,I,0.0,NaN,NaN,NaN,34999.0,39062.0,0.0,1.06480,C
2,206999.0,20260715.C00120,2019-06-25,99.371094,99.472656,99.421875,M,I,0.0,NaN,NaN,NaN,34999.0,39062.0,0.0,1.06499,C
3,206999.0,20260715.C00120,2019-06-26,99.144531,99.246094,99.195312,M,I,0.0,NaN,NaN,NaN,34999.0,39062.0,0.0,1.06517,C
4,206999.0,20260715.C00120,2019-06-27,99.230469,99.332031,99.281250,M,I,0.0,NaN,NaN,NaN,34999.0,39062.0,0.0,1.06536,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,207431.0,20241015.C00120,2021-01-25,108.035156,108.093750,108.064453,M,I,0.0,NaN,NaN,NaN,31724.0,35713.0,0.0,1.01442,C
3518,207431.0,20241015.C00120,2021-01-26,108.007812,108.066406,108.037109,M,I,0.0,NaN,NaN,NaN,31724.0,35713.0,0.0,1.01440,C
3519,207431.0,20241015.C00120,2021-01-27,108.000000,108.058594,108.029297,M,I,0.0,NaN,NaN,NaN,31724.0,35713.0,0.0,1.01438,C
3520,207431.0,20241015.C00120,2021-01-28,107.960938,108.019531,107.990234,M,I,0.0,NaN,NaN,NaN,31724.0,35713.0,0.0,1.01436,C


# Save to Excel

In [8]:
outfile = f'../data/treasury_quotes_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=False)

In [28]:
outfile = f'../data/treasury_ts_{MATDT}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    ts.to_excel(writer, sheet_name= 'ts', index=False)

In [29]:
outfile = f'../data/treasury_ts_issue_{KEYNO_ts}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    ts_issue.to_excel(writer, sheet_name= 'ts', index=False)

In [34]:
outfile = f'../data/treasury_ts_issue_TIPS_{KEYNO_tips}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    ts_issue_tips.to_excel(writer, sheet_name= 'ts', index=False)

In [12]:
outfile = f'../data/yields.xlsx'
with pd.ExcelWriter(outfile) as writer:
    data_ft.to_excel(writer, sheet_name= 'yields', index=True)